In [1]:
import pandas as pd

df = pd.read_feather('python_comm.feather')
df.shape
df.head()

,review_comments,diff_hunk,path,repo
0,"Actually, these methods are deprecated on 2.0....","@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue
1,I think we no longer need namespaces for decla...,"@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue
2,`[key: string]: any` is dangerous to put this ...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue
3,Directives no longer have instances and we don...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue
4,ComputedOptions may have `cache: boolean` prop...,"@@ -0,0 +1,71 @@\n+import { Vue } from ""./vue....",types/options.d.ts,vuejs/vue


In [2]:
df['row_id'] = df.index
df.head()

,review_comments,diff_hunk,path,repo,row_id
0,"Actually, these methods are deprecated on 2.0....","@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue,0
1,I think we no longer need namespaces for decla...,"@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue,1
2,`[key: string]: any` is dangerous to put this ...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue,2
3,Directives no longer have instances and we don...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue,3
4,ComputedOptions may have `cache: boolean` prop...,"@@ -0,0 +1,71 @@\n+import { Vue } from ""./vue....",types/options.d.ts,vuejs/vue,4


In [3]:
review_len = df['review_comments'].str.len().mean()
hunk_len = df['diff_hunk'].str.len().mean()

print(f'review_len: {review_len}, hunk_len: {hunk_len}')

review_len: 162.11531264638978, hunk_len: 1316.8491100150395


In [4]:
sample = df['diff_hunk']

In [5]:
diff = df['diff_hunk']
diff.head()

0    @@ -0,0 +1,127 @@\n+interface Array<T> {\n+  $...
1    @@ -0,0 +1,127 @@\n+interface Array<T> {\n+  $...
2    @@ -0,0 +1,122 @@\n+type Constructor = {\n+  n...
3    @@ -0,0 +1,122 @@\n+type Constructor = {\n+  n...
4    @@ -0,0 +1,71 @@\n+import { Vue } from "./vue....
Name: diff_hunk, dtype: object

In [6]:
def filter_diff_headers(sample_text):
    filtered = []
    for line in sample_text.splitlines():
        if line.startswith(('@@', '+++', '---', 'diff')):
            continue
        filtered.append(line)
    return "\n".join(filtered)

In [7]:
def group_changed_indices(indices,total_lines):
    if not indices:
        return []
    
    groups = []
    current_group = [indices[0]]

    line_count = total_lines

    if line_count <= 15:
        threshold = 3
    elif line_count <= 40:
        threshold = 5
    else:
        threshold = 7

    for i in range(1, len(indices)):
        if indices[i] - indices[i-1] <= threshold:
            current_group.append(indices[i])
        else:
            groups.append(current_group)
            current_group = [indices[i]]
    groups.append(current_group)
    return groups

In [8]:
all_chunks = []
for row_id, row in df.iterrows():          # ← SUPER IMPORTANT NEW LOOP
    sample = [row['diff_hunk']]  
    for i,samp in enumerate(sample):
        cleaned_samp = filter_diff_headers(samp)
    
        lines = [ln.rstrip() for ln in cleaned_samp.split('\n')]
        changed_indices = []
        for idx,line in enumerate(lines):
            if line.startswith(('+','-')) and not line.startswith(('+++','---')):
                changed_indices.append(idx)
                
        # print(f"Hunk {i+1} changed indices: {changed_indices}")
        groups = group_changed_indices(changed_indices, len(lines))
        
        # print(f"Chunk {i+1} grouped:", groups)
    
    for g in groups:
            start, end = g[0], g[-1]
            chunk_lines = lines[start:end+1]
            all_chunks.append({
                "row_id": row_id,
                "hunk_id": i + 1,
                "chunk_id": f"{i+1}_{start}-{end}",
                "chunk_text": "\n".join(chunk_lines),
                "num_lines": len(chunk_lines)
            })

df_chunks = pd.DataFrame(all_chunks)

In [9]:
df_chunks.shape

(1657015, 5)

In [10]:
df_chunks.to_feather("CC.feather")

In [11]:
df_cc = pd.read_feather('CC.feather')

In [12]:
df_cc.head()

,row_id,hunk_id,chunk_id,chunk_text,num_lines
0,0,1,1_0-3,+interface Array<T> {\n+ $remove(item: T): Ar...,4
1,1,1,1_0-57,+interface Array<T> {\n+ $remove(item: T): Ar...,58
2,2,1,1_0-83,+type Constructor = {\n+ new (...args: any[])...,84
3,3,1,1_0-91,+type Constructor = {\n+ new (...args: any[])...,92
4,4,1,1_0-48,"+import { Vue } from ""./vue.d"";\n+import { VNo...",49


In [13]:
comments_df = df[df['review_comments'].notna()].copy()

In [14]:
def clean_comment_text(text):
    if not isinstance(text,str):
        return ""
    text = re.sub(r'```.*?```', '', text, flags=re.S)   # remove code blocks
    text = re.sub(r'\s+', ' ', text).strip()            # collapse spaces
    return text

    comments_df['cleaned_comments'] = comments_df['review_comments'].apply(clean_comment_text)

In [15]:
comments_df.to_feather('cleaned_cc.feather')

In [16]:
clean_df = pd.read_feather('cleaned_cc.feather')
clean_df.head()

,review_comments,diff_hunk,path,repo,row_id
0,"Actually, these methods are deprecated on 2.0....","@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue,0
1,I think we no longer need namespaces for decla...,"@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue,1
2,`[key: string]: any` is dangerous to put this ...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue,2
3,Directives no longer have instances and we don...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue,3
4,ComputedOptions may have `cache: boolean` prop...,"@@ -0,0 +1,71 @@\n+import { Vue } from ""./vue....",types/options.d.ts,vuejs/vue,4


In [17]:
clean_df.columns

Index(['review_comments', 'diff_hunk', 'path', 'repo', 'row_id'], dtype='object')

In [18]:
df_cc.columns

Index(['row_id', 'hunk_id', 'chunk_id', 'chunk_text', 'num_lines'], dtype='object')

In [19]:
df_merged = pd.merge(clean_df, df_cc,
                       on = 'row_id',
                       how = 'left')

In [20]:
df_merged.head()

,review_comments,diff_hunk,path,repo,row_id,hunk_id,chunk_id,chunk_text,num_lines
0,"Actually, these methods are deprecated on 2.0....","@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue,0,1.0,1_0-3,+interface Array<T> {\n+ $remove(item: T): Ar...,4.0
1,I think we no longer need namespaces for decla...,"@@ -0,0 +1,127 @@\n+interface Array<T> {\n+ $...",types/index.d.ts,vuejs/vue,1,1.0,1_0-57,+interface Array<T> {\n+ $remove(item: T): Ar...,58.0
2,`[key: string]: any` is dangerous to put this ...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue,2,1.0,1_0-83,+type Constructor = {\n+ new (...args: any[])...,84.0
3,Directives no longer have instances and we don...,"@@ -0,0 +1,122 @@\n+type Constructor = {\n+ n...",types/index.d.ts,vuejs/vue,3,1.0,1_0-91,+type Constructor = {\n+ new (...args: any[])...,92.0
4,ComputedOptions may have `cache: boolean` prop...,"@@ -0,0 +1,71 @@\n+import { Vue } from ""./vue....",types/options.d.ts,vuejs/vue,4,1.0,1_0-48,"+import { Vue } from ""./vue.d"";\n+import { VNo...",49.0


In [21]:
df_merged.to_feather('merged.feather')

In [22]:
df_merged.shape

(1675291, 9)